In [46]:
import pandas as pd
import scipy
import seaborn as sns
%matplotlib inline
from scipy import stats
import random
import netaddr

In [47]:
df = pd.read_csv('~/all_portal_data_cleaned.csv')

In [48]:
df['firsttime'] = pd.to_datetime(df['firsttime'])
df['lasttime'] = pd.to_datetime(df['lasttime'])

In [49]:
# drops 1 row
df = df.dropna(subset=['inetnum'])
date_ranges = pd.date_range(start='2011-08', end='2017-04', freq='M')
print(date_ranges)

DatetimeIndex(['2011-08-31', '2011-09-30', '2011-10-31', '2011-11-30',
               '2011-12-31', '2012-01-31', '2012-02-29', '2012-03-31',
               '2012-04-30', '2012-05-31', '2012-06-30', '2012-07-31',
               '2012-08-31', '2012-09-30', '2012-10-31', '2012-11-30',
               '2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31',
               '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31',
               '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30',
               '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31',
               '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31',
               '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30',
               '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31',
               '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31',
               '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30',
               '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31',
      

In [50]:
country_df = pd.DataFrame(index=df['country'].unique())

In [51]:
country_df

""
LK
US
PK
TR
CL
GE
IT
AR
EU
MA


In [52]:
for i in range(len(date_ranges) - 1):
    start_month = date_ranges[i]
    end_month = date_ranges[i+1]
    counts = df.loc[[start < end_month and end > start_month for start, end in zip(df['firsttime'], df['lasttime'])]].groupby('country').count()['id']
    for asys in country_df.index:
        country_df.loc[asys, str(end_month.year) + '-' + str(end_month.month)] = (counts[asys] if asys in counts.index else 0)

In [53]:
for asys in country_df.index:
    size = 0
    inetnums = df.loc[df['country'] == asys, 'inetnum'].unique()
    
    for iprange in inetnums:
        iprange = iprange.split('-')
        start_ip = iprange[0].strip(' ()')
        end_ip = iprange[1].strip(' ()')
        
        added_size = netaddr.iprange_to_cidrs(start_ip, end_ip)[0].size
        size += added_size
        
    country_df.loc[asys, 'size'] = size

In [54]:
def normalize_col(x, colname:str):
    x[colname + ' normalized'] = x[colname] / x['size']
    #print(x[[colname, colname + ' normalized']])
    return x

In [55]:
cols = country_df.columns
for col in cols:
    country_df = country_df.apply(normalize_col, axis=1, colname=col)

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


In [69]:
country_df

,2011-9,2011-10,2011-11,2011-12,2012-1,2012-2,2012-3,2012-4,2012-5,2012-6,...,2016-7 normalized,2016-8 normalized,2016-9 normalized,2016-10 normalized,2016-11 normalized,2016-12 normalized,2017-1 normalized,2017-2 normalized,2017-3 normalized,size normalized
LK,1.0,3.0,3.0,5.0,5.0,19.0,56.0,4.0,4.0,6.0,...,3.481021e-05,2.784817e-05,2.784817e-05,2.784817e-05,2.784817e-05,2.784817e-05,2.784817e-05,2.784817e-05,2.784817e-05,1.0
US,91.0,2816.0,7985.0,16582.0,20101.0,28687.0,40243.0,21095.0,18278.0,25127.0,...,3.149701e-05,3.011732e-05,2.985462e-05,3.418806e-05,3.145537e-05,3.301019e-05,3.351637e-05,3.385382e-05,3.384741e-05,1.0
PK,1.0,2.0,2.0,85.0,88.0,94.0,108.0,27.0,17.0,30.0,...,1.045823e-05,1.126271e-05,8.849273e-06,8.044793e-06,8.849273e-06,8.849273e-06,8.849273e-06,8.849273e-06,8.849273e-06,1.0
TR,7.0,141.0,343.0,553.0,808.0,1083.0,1406.0,811.0,625.0,1027.0,...,1.526157e-04,1.663396e-04,1.645207e-04,1.805594e-04,2.364469e-04,3.095305e-04,3.340019e-04,3.082077e-04,3.177978e-04,1.0
CL,1.0,13.0,94.0,123.0,388.0,413.0,726.0,381.0,256.0,489.0,...,4.399940e-05,3.823757e-05,3.299955e-05,3.247575e-05,3.247575e-05,2.880913e-05,2.880913e-05,3.090434e-05,4.504700e-05,1.0
GE,2.0,3.0,5.0,7.0,38.0,51.0,109.0,69.0,66.0,79.0,...,2.280023e-04,2.280023e-04,2.280023e-04,2.553626e-04,2.462425e-04,2.918430e-04,2.736028e-04,2.644827e-04,2.371224e-04,1.0
IT,12.0,66.0,379.0,671.0,735.0,845.0,1139.0,755.0,662.0,1036.0,...,3.832054e-05,4.031337e-05,4.222317e-05,4.110220e-05,4.222317e-05,4.309504e-05,4.467270e-05,4.467270e-05,4.699767e-05,1.0
AR,1.0,22.0,79.0,106.0,168.0,203.0,349.0,165.0,143.0,284.0,...,1.893708e-05,1.863408e-05,1.833109e-05,1.742211e-05,1.727061e-05,1.696762e-05,1.666463e-05,1.772510e-05,1.817959e-05,1.0
EU,1.0,14.0,28.0,56.0,78.0,95.0,106.0,88.0,77.0,148.0,...,3.686483e-08,3.802411e-08,3.756040e-08,3.779225e-08,3.756040e-08,4.011079e-08,4.034265e-08,3.941523e-08,4.011079e-08,1.0
MA,2.0,75.0,136.0,248.0,351.0,403.0,465.0,258.0,263.0,297.0,...,3.227670e-05,3.737302e-05,6.115585e-05,6.625217e-05,6.625217e-05,6.625217e-05,9.003499e-05,7.644481e-05,7.814358e-05,1.0


In [57]:
country_df.to_csv('~/country_data.csv')

In [70]:
normalized_df = country_df.loc[:, [col for col in country_df.columns if col.find('normalized') > 0 or col == 'size']]

In [72]:
normalized_df.drop(columns=['size', 'size normalized'], inplace=True)

In [85]:
cols = list(normalized_df)

In [93]:
normalized_df['total'] = normalized_df[cols].sum(axis=1)/len(normalized_df['total'])
normalized_df['total'][0:10]

LK    8.399080e-05
US    2.383285e-05
PK    2.119494e-05
TR    1.537244e-04
CL    1.240270e-04
GE    2.987415e-04
IT    5.076617e-05
AR    4.745922e-05
EU    2.621440e-08
MA    2.667728e-04
Name: total, dtype: float64

In [94]:
normalized_df['country'] = normalized_df.index
cols = ['country', 'total']
country_index = normalized_df[cols].reset_index().drop(columns='index')
country_index.head()

,country,total
0,LK,0.000084
1,US,0.000024
2,PK,0.000021
3,TR,0.000154
4,CL,0.000124


In [95]:
country_index.to_csv('~/country_index.csv')